# Complete End-to-End Reproduction

This notebook reproduces all results from the paper:

> **"Delta Observer: Learning Continuous Semantic Manifolds Between Neural Network Representations"**  
> Aaron (Tripp) Josserand-Austin | EntroMorphic Research Team  
> [OSF MetaArXiv](https://doi.org/10.17605/OSF.IO/CNJTP)

---

## Key Discovery: Transient Clustering

**Clustering is scaffolding, not structure.** Networks build geometric organization to *learn* semantic concepts, then discard that organization once the concepts are encoded in the weights.

| Training Phase | R² | Silhouette | Interpretation |
|----------------|-----|-----------|----------------|
| Early (epoch 0) | 0.36 | -0.02 | Random initialization |
| Learning (epoch 20) | 0.94 | **0.33** | Clustering emerges |
| Final (epoch 200) | 0.99 | -0.02 | Clustering dissolves |

---

## Pipeline Overview

1. **Generate Dataset** - All 512 possible 4-bit + 4-bit additions
2. **Online Training** - Train all three models concurrently (Monolithic, Compositional, Delta Observer)
3. **Trajectory Analysis** - Track R² and Silhouette during training
4. **Discover Transient Clustering** - Observe clustering emerge then dissolve
5. **Generate Figures** - Reproduce paper visualizations

**Estimated runtime:** ~15 minutes on CPU, ~5 minutes on GPU

---

## Setup & Configuration

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from tqdm import tqdm
import os

# Try UMAP, fall back to PCA
try:
    from umap import UMAP
    HAS_UMAP = True
except ImportError:
    HAS_UMAP = False
    print("UMAP not available, using PCA for visualization")

# Configuration
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
RANDOM_SEED = 42
EPOCHS = 200
BATCH_SIZE = 64
LEARNING_RATE = 0.001
LATENT_DIM = 16
SNAPSHOT_INTERVAL = 5  # Save latent space every N epochs

# Set seeds
torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

# Create directories
os.makedirs('../data', exist_ok=True)
os.makedirs('../figures', exist_ok=True)

print(f"Device: {DEVICE}")
print(f"Training for {EPOCHS} epochs")
print(f"Snapshot interval: every {SNAPSHOT_INTERVAL} epochs")
print("Configuration complete!")

## Step 1: Generate 4-bit Addition Dataset

In [ ]:
def generate_4bit_addition_dataset():
    """Generate all 512 possible 4-bit + 4-bit additions."""
    inputs = []
    outputs = []
    carry_counts = []
    
    for a in range(16):
        for b in range(16):
            # Input: [a0, a1, a2, a3, b0, b1, b2, b3]
            a_bits = [(a >> i) & 1 for i in range(4)]
            b_bits = [(b >> i) & 1 for i in range(4)]
            input_bits = a_bits + b_bits
            
            # Output: 5-bit sum
            sum_val = a + b
            output_bits = [(sum_val >> i) & 1 for i in range(5)]
            
            # Carry count
            carry = 0
            count = 0
            for i in range(4):
                bit_sum = a_bits[i] + b_bits[i] + carry
                if bit_sum >= 2:
                    count += 1
                    carry = 1
                else:
                    carry = 0
            
            inputs.append(input_bits)
            outputs.append(output_bits)
            carry_counts.append(count)
    
    return (np.array(inputs, dtype=np.float32), 
            np.array(outputs, dtype=np.float32),
            np.array(carry_counts, dtype=np.int64))

X, y, carry_counts = generate_4bit_addition_dataset()
print(f"Dataset: {X.shape[0]} examples")
print(f"Input: {X.shape[1]} bits, Output: {y.shape[1]} bits")
print(f"Carry count distribution: {np.bincount(carry_counts)}")

In [ ]:
class AdditionDataset(Dataset):
    def __init__(self, X, y, carry_counts):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
        self.carry_counts = torch.tensor(carry_counts, dtype=torch.long)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx], self.carry_counts[idx]

dataset = AdditionDataset(X, y, carry_counts)
train_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
full_loader = DataLoader(dataset, batch_size=512, shuffle=False)
print("Dataset ready")

## Step 2: Define Model Architectures

In [ ]:
class MonolithicMLP(nn.Module):
    """Standard MLP that processes all bits at once."""
    def __init__(self, hidden_dim=64):
        super().__init__()
        self.fc1 = nn.Linear(8, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, 5)
        self.hidden_dim = hidden_dim
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        hidden = torch.relu(self.fc2(x))
        out = torch.sigmoid(self.fc3(hidden))
        return out, hidden


class CompositionalNetwork(nn.Module):
    """Modular network with separate per-bit processing."""
    def __init__(self, module_dim=16):
        super().__init__()
        self.bit_modules = nn.ModuleList([
            nn.Sequential(
                nn.Linear(3, module_dim),
                nn.ReLU(),
                nn.Linear(module_dim, module_dim),
                nn.ReLU()
            ) for _ in range(4)
        ])
        self.output = nn.Linear(4 * module_dim, 5)
        self.hidden_dim = 4 * module_dim
    
    def forward(self, x):
        batch_size = x.size(0)
        bit_outputs = []
        carry = torch.zeros(batch_size, 1, device=x.device)
        
        for i in range(4):
            a_bit = x[:, i:i+1]
            b_bit = x[:, i+4:i+5]
            module_input = torch.cat([a_bit, b_bit, carry], dim=1)
            module_output = self.bit_modules[i](module_input)
            bit_outputs.append(module_output)
            carry = torch.sigmoid(module_output[:, :1])
        
        hidden = torch.cat(bit_outputs, dim=1)
        out = torch.sigmoid(self.output(hidden))
        return out, hidden


class DeltaObserver(nn.Module):
    """Learns shared latent space between two architectures."""
    def __init__(self, mono_dim=64, comp_dim=64, latent_dim=16):
        super().__init__()
        self.mono_encoder = nn.Sequential(
            nn.Linear(mono_dim, 32), nn.ReLU(), nn.Dropout(0.1)
        )
        self.comp_encoder = nn.Sequential(
            nn.Linear(comp_dim, 32), nn.ReLU(), nn.Dropout(0.1)
        )
        self.shared_encoder = nn.Sequential(
            nn.Linear(64, 32), nn.ReLU(), nn.Dropout(0.1),
            nn.Linear(32, latent_dim)
        )
        self.mono_decoder = nn.Sequential(
            nn.Linear(latent_dim, 32), nn.ReLU(),
            nn.Linear(32, mono_dim)
        )
        self.comp_decoder = nn.Sequential(
            nn.Linear(latent_dim, 32), nn.ReLU(),
            nn.Linear(32, comp_dim)
        )
        self.carry_head = nn.Sequential(
            nn.Linear(latent_dim, 8), nn.ReLU(),
            nn.Linear(8, 1)
        )
        self.latent_dim = latent_dim
    
    def encode(self, mono_act, comp_act):
        mono_enc = self.mono_encoder(mono_act)
        comp_enc = self.comp_encoder(comp_act)
        joint = torch.cat([mono_enc, comp_enc], dim=-1)
        return self.shared_encoder(joint)
    
    def forward(self, mono_act, comp_act):
        latent = self.encode(mono_act, comp_act)
        return {
            'latent': latent,
            'mono_recon': self.mono_decoder(latent),
            'comp_recon': self.comp_decoder(latent),
            'carry_pred': self.carry_head(latent)
        }

print("Model architectures defined")

## Step 3: Online Training (All Models Concurrently)

**This is the key innovation.** The Delta Observer watches training as it happens, capturing temporal dynamics invisible to post-hoc analysis.

In [ ]:
# Initialize models
mono_model = MonolithicMLP(hidden_dim=64).to(DEVICE)
comp_model = CompositionalNetwork(module_dim=16).to(DEVICE)
delta_model = DeltaObserver(mono_dim=64, comp_dim=64, latent_dim=LATENT_DIM).to(DEVICE)

# Optimizers
mono_opt = optim.Adam(mono_model.parameters(), lr=LEARNING_RATE)
comp_opt = optim.Adam(comp_model.parameters(), lr=LEARNING_RATE)
delta_opt = optim.Adam(delta_model.parameters(), lr=LEARNING_RATE)

criterion = nn.BCELoss()

# Trajectory storage
trajectory = {
    'epochs': [],
    'latents': [],
    'carry_counts': [],
    'mono_acc': [],
    'comp_acc': [],
    'r2': [],
    'silhouette': []
}

print("Models initialized")
print(f"Monolithic: {sum(p.numel() for p in mono_model.parameters())} params")
print(f"Compositional: {sum(p.numel() for p in comp_model.parameters())} params")
print(f"Delta Observer: {sum(p.numel() for p in delta_model.parameters())} params")

In [ ]:
def compute_metrics(latents, carry_counts):
    """Compute R² and Silhouette score."""
    reg = LinearRegression()
    reg.fit(latents, carry_counts)
    r2 = r2_score(carry_counts, reg.predict(latents))
    
    try:
        sil = silhouette_score(latents, carry_counts)
    except:
        sil = 0.0
    
    return r2, sil


def compute_accuracy(model, loader):
    """Compute model accuracy."""
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets, _ in loader:
            inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
            outputs, _ = model(inputs)
            pred_bits = (outputs > 0.5).float()
            correct += (pred_bits == targets).all(dim=1).sum().item()
            total += inputs.size(0)
    return 100 * correct / total


def snapshot_latents(mono_model, comp_model, delta_model, loader, device):
    """Extract latent representations for all samples."""
    mono_model.eval()
    comp_model.eval()
    delta_model.eval()
    
    all_latents = []
    all_carry = []
    
    with torch.no_grad():
        for inputs, _, carry in loader:
            inputs = inputs.to(device)
            _, mono_h = mono_model(inputs)
            _, comp_h = comp_model(inputs)
            latent = delta_model.encode(mono_h, comp_h)
            all_latents.append(latent.cpu().numpy())
            all_carry.append(carry.numpy())
    
    return np.concatenate(all_latents), np.concatenate(all_carry)

In [ ]:
print("="*70)
print("ONLINE TRAINING - All models train concurrently")
print("="*70)
print("\nThe Delta Observer watches training as it happens...\n")

for epoch in tqdm(range(EPOCHS), desc="Training"):
    mono_model.train()
    comp_model.train()
    delta_model.train()
    
    for inputs, targets, carry in train_loader:
        inputs = inputs.to(DEVICE)
        targets = targets.to(DEVICE)
        carry = carry.to(DEVICE).float()
        
        # --- Train Monolithic ---
        mono_opt.zero_grad()
        mono_out, mono_h = mono_model(inputs)
        mono_loss = criterion(mono_out, targets)
        mono_loss.backward()
        mono_opt.step()
        
        # --- Train Compositional ---
        comp_opt.zero_grad()
        comp_out, comp_h = comp_model(inputs)
        comp_loss = criterion(comp_out, targets)
        comp_loss.backward()
        comp_opt.step()
        
        # --- Train Delta Observer (detached activations) ---
        with torch.no_grad():
            _, mono_h_det = mono_model(inputs)
            _, comp_h_det = comp_model(inputs)
        
        delta_opt.zero_grad()
        delta_out = delta_model(mono_h_det.detach(), comp_h_det.detach())
        
        recon_loss = (nn.functional.mse_loss(delta_out['mono_recon'], mono_h_det.detach()) +
                      nn.functional.mse_loss(delta_out['comp_recon'], comp_h_det.detach()))
        carry_loss = nn.functional.mse_loss(delta_out['carry_pred'].squeeze(), carry)
        delta_loss = recon_loss + 0.1 * carry_loss
        delta_loss.backward()
        delta_opt.step()
    
    # --- Snapshot at intervals ---
    if epoch % SNAPSHOT_INTERVAL == 0 or epoch == EPOCHS - 1:
        latents, carries = snapshot_latents(mono_model, comp_model, delta_model, full_loader, DEVICE)
        r2, sil = compute_metrics(latents, carries)
        mono_acc = compute_accuracy(mono_model, full_loader)
        comp_acc = compute_accuracy(comp_model, full_loader)
        
        trajectory['epochs'].append(epoch)
        trajectory['latents'].append(latents.copy())
        trajectory['carry_counts'].append(carries.copy())
        trajectory['r2'].append(r2)
        trajectory['silhouette'].append(sil)
        trajectory['mono_acc'].append(mono_acc)
        trajectory['comp_acc'].append(comp_acc)
        
        if epoch % 20 == 0:
            print(f"\nEpoch {epoch:3d}: R²={r2:.4f}, Sil={sil:.4f}, Mono={mono_acc:.1f}%, Comp={comp_acc:.1f}%")

print("\n" + "="*70)
print("✅ Online training complete!")
print("="*70)

## Step 4: Discover Transient Clustering

**The key finding:** Clustering peaks during training then dissolves.

In [ ]:
epochs = np.array(trajectory['epochs'])
r2_values = np.array(trajectory['r2'])
sil_values = np.array(trajectory['silhouette'])

# Find peak clustering
peak_idx = np.argmax(sil_values)
peak_epoch = epochs[peak_idx]
peak_sil = sil_values[peak_idx]

print("="*70)
print("TRANSIENT CLUSTERING DISCOVERY")
print("="*70)
print(f"\nPeak clustering: Silhouette = {peak_sil:.4f} at epoch {peak_epoch}")
print(f"Final state:     Silhouette = {sil_values[-1]:.4f} at epoch {epochs[-1]}")
print(f"\nFinal R² (accessibility): {r2_values[-1]:.4f}")
print("\n" + "-"*70)
print("INTERPRETATION")
print("-"*70)
print("\nClustering EMERGES during learning (scaffolding)")
print("Clustering DISSOLVES after convergence (scaffolding removed)")
print("\nThe semantic primitive is in the TRAJECTORY, not the final state.")
print("="*70)

In [ ]:
# Plot: Transient Clustering
fig, ax1 = plt.subplots(figsize=(12, 6))

color1 = '#2ecc71'  # Green for R²
color2 = '#e74c3c'  # Red for Silhouette

ax1.set_xlabel('Training Epoch', fontsize=12)
ax1.set_ylabel('R² (Linear Accessibility)', color=color1, fontsize=12)
line1, = ax1.plot(epochs, r2_values, color=color1, linewidth=2.5, marker='o', markersize=4, label='R²')
ax1.tick_params(axis='y', labelcolor=color1)
ax1.set_ylim(0, 1.05)
ax1.axhline(y=0.9, color=color1, linestyle='--', alpha=0.3)

ax2 = ax1.twinx()
ax2.set_ylabel('Silhouette Score (Clustering)', color=color2, fontsize=12)
line2, = ax2.plot(epochs, sil_values, color=color2, linewidth=2.5, marker='s', markersize=4, label='Silhouette')
ax2.tick_params(axis='y', labelcolor=color2)
ax2.set_ylim(-0.1, 0.5)
ax2.axhline(y=0, color=color2, linestyle='--', alpha=0.3)

# Annotate peak
ax2.annotate(f'Peak: {peak_sil:.2f}\n(epoch {peak_epoch})',
             xy=(peak_epoch, peak_sil),
             xytext=(peak_epoch + 30, peak_sil + 0.08),
             fontsize=10,
             arrowprops=dict(arrowstyle='->', color=color2, alpha=0.7),
             color=color2)

ax1.set_title('Transient Clustering: Scaffolding Emerges Then Dissolves', fontsize=14, fontweight='bold')
ax1.legend([line1, line2], ['R² (Accessibility)', 'Silhouette (Clustering)'], loc='center right')

plt.tight_layout()
plt.savefig('../figures/figure5_training_curves.png', dpi=150, bbox_inches='tight')
plt.show()
print("✅ Figure 5 (Training Curves) saved")

## Step 5: Visualize Final Latent Space

In [ ]:
# Get final latent space
final_latents = trajectory['latents'][-1]
final_carry = trajectory['carry_counts'][-1]

# Dimensionality reduction
if HAS_UMAP:
    reducer = UMAP(n_components=2, random_state=RANDOM_SEED, n_neighbors=15, min_dist=0.1)
    latents_2d = reducer.fit_transform(final_latents)
    method = "UMAP"
else:
    reducer = PCA(n_components=2, random_state=RANDOM_SEED)
    latents_2d = reducer.fit_transform(final_latents)
    method = "PCA"

# Final metrics
final_r2, final_sil = compute_metrics(final_latents, final_carry)

# Plot
fig, ax = plt.subplots(figsize=(10, 8))
scatter = ax.scatter(latents_2d[:, 0], latents_2d[:, 1],
                     c=final_carry, cmap='viridis',
                     s=50, alpha=0.7, edgecolors='white', linewidth=0.5)

cbar = plt.colorbar(scatter, ax=ax, label='Carry Count')
cbar.set_ticks([0, 1, 2, 3, 4])

ax.set_xlabel(f'{method} Dimension 1', fontsize=12)
ax.set_ylabel(f'{method} Dimension 2', fontsize=12)
ax.set_title('Online Delta Observer Latent Space\n(Final State)', fontsize=14, fontweight='bold')

# Add metrics
textstr = f'R² = {final_r2:.4f}\nSilhouette = {final_sil:.4f}'
props = dict(boxstyle='round', facecolor='white', alpha=0.8)
ax.text(0.02, 0.98, textstr, transform=ax.transAxes, fontsize=11,
        verticalalignment='top', bbox=props)

plt.tight_layout()
plt.savefig('../figures/figure2_delta_latent_space.png', dpi=150, bbox_inches='tight')
plt.show()
print("✅ Figure 2 (Latent Space) saved")

## Step 6: Compare Methods

In [ ]:
# Get activations for PCA baseline comparison
mono_model.eval()
comp_model.eval()

with torch.no_grad():
    all_inputs = torch.tensor(X, dtype=torch.float32).to(DEVICE)
    _, mono_h = mono_model(all_inputs)
    _, comp_h = comp_model(all_inputs)
    mono_act = mono_h.cpu().numpy()
    comp_act = comp_h.cpu().numpy()

# PCA baseline
combined = np.concatenate([mono_act, comp_act], axis=1)
pca = PCA(n_components=LATENT_DIM, random_state=RANDOM_SEED)
pca_latents = pca.fit_transform(combined)
pca_r2, pca_sil = compute_metrics(pca_latents, carry_counts)

print("="*70)
print("METHOD COMPARISON")
print("="*70)
print(f"\n{'Method':<25} {'R²':>10} {'Silhouette':>12} {'Δ vs PCA':>10}")
print("-"*60)
print(f"{'Online Observer':<25} {final_r2:>10.4f} {final_sil:>12.4f} {(final_r2-pca_r2)*100:>+9.1f}%")
print(f"{'PCA Baseline':<25} {pca_r2:>10.4f} {pca_sil:>12.4f} {'---':>10}")
print("="*70)

In [ ]:
# Bar chart comparison
fig, ax = plt.subplots(figsize=(8, 6))

methods = ['Online\nObserver', 'PCA\nBaseline']
r2_vals = [final_r2, pca_r2]
colors = ['#3498db', '#95a5a6']

bars = ax.bar(methods, r2_vals, color=colors, edgecolor='black', linewidth=1.5)

for bar, val in zip(bars, r2_vals):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.005,
            f'{val:.4f}', ha='center', va='bottom', fontsize=12, fontweight='bold')

ax.set_ylabel('R² (Linear Accessibility)', fontsize=12)
ax.set_title('Online Observation vs PCA Baseline', fontsize=14, fontweight='bold')
ax.set_ylim(0.9, 1.01)
ax.axhline(y=pca_r2, color='gray', linestyle='--', alpha=0.5)

# Delta annotation
delta_pct = (final_r2 - pca_r2) * 100
ax.text(0.5, 0.95, f'+{delta_pct:.1f}%', ha='center', fontsize=14, fontweight='bold', color='green')

plt.tight_layout()
plt.savefig('../figures/figure_method_comparison.png', dpi=150, bbox_inches='tight')
plt.show()
print("✅ Method comparison figure saved")

## Step 7: Save Results

In [ ]:
# Save trajectory data
np.savez('../data/online_observer_trajectory.npz',
         snapshots=np.array(trajectory['latents']),
         epochs=np.array(trajectory['epochs']))

# Save final latents
np.savez('../data/online_observer_latents.npz',
         latents=final_latents,
         carry_counts=final_carry,
         mono_activations=mono_act,
         comp_activations=comp_act,
         bit_positions=np.zeros_like(final_carry))  # placeholder

print("✅ Data saved to ../data/")

## Final Summary

In [ ]:
print("\n" + "="*70)
print("REPRODUCTION COMPLETE")
print("="*70)

print("\n📊 MODELS TRAINED (Online, Concurrently)")
print(f"  Monolithic MLP: {trajectory['mono_acc'][-1]:.1f}% accuracy")
print(f"  Compositional Network: {trajectory['comp_acc'][-1]:.1f}% accuracy")

print("\n🎯 KEY DISCOVERY: TRANSIENT CLUSTERING")
print(f"  Peak clustering: Silhouette = {peak_sil:.4f} at epoch {peak_epoch}")
print(f"  Final state:     Silhouette = {final_sil:.4f}")
print(f"  Final R²:        {final_r2:.4f}")

print("\n📈 METHOD COMPARISON")
print(f"  Online Observer: R² = {final_r2:.4f}")
print(f"  PCA Baseline:    R² = {pca_r2:.4f}")
print(f"  Improvement:     +{(final_r2-pca_r2)*100:.1f}%")

print("\n📁 FILES GENERATED")
print("  Data:")
print("    - data/online_observer_trajectory.npz")
print("    - data/online_observer_latents.npz")
print("  Figures:")
print("    - figures/figure2_delta_latent_space.png")
print("    - figures/figure5_training_curves.png")
print("    - figures/figure_method_comparison.png")

print("\n" + "="*70)
print("KEY INSIGHT")
print("="*70)
print("\n  CLUSTERING IS SCAFFOLDING, NOT STRUCTURE.")
print("\n  Networks build geometric organization to LEARN,")
print("  then DISCARD it once concepts are encoded in weights.")
print("\n  The semantic primitive is in the TRAJECTORY,")
print("  not the final representation.")
print("\n" + "="*70)
print("\n✅ All results successfully reproduced!")